In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya_speech.train.model.conformer as conformer
import malaya_speech.train.model.transducer as transducer
import malaya_speech
import tensorflow as tf
import numpy as np
import json
from glob import glob
from sklearn.utils import shuffle






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
subwords_malay = malaya_speech.subword.load('transducer.subword')
subwords_singlish = malaya_speech.subword.load('transducer-singlish.subword')
langs = [subwords_malay, subwords_singlish]
len_vocab = [l.vocab_size for l in langs]
singlish_directories = ['WAVE', 'part3-splitted', 'CHANNEL0']

In [4]:
featurizer = malaya_speech.tf_featurization.STTFeaturizer(
    normalize_per_feature = True
)

In [5]:
n_mels = 80
sr = 16000
maxlen = 18
minlen_text = 1

def mp3_to_wav(file, sr = sr):
    audio = AudioSegment.from_file(file)
    audio = audio.set_frame_rate(sr).set_channels(1)
    sample = np.array(audio.get_array_of_samples())
    return malaya_speech.astype.int_to_float(sample), sr


def generate(file):
    with open(file) as fopen:
        dataset = json.load(fopen)
    audios, cleaned_texts = dataset['X'], dataset['Y']
    # audios, cleaned_texts = shuffle(audios, cleaned_texts)
    for i in range(len(audios)):
        try:
            if not os.path.exists(audios[i]):
                continue
            if any([d in audios[i] for d in singlish_directories]):
                lang = 2
            else:
                lang = 1
                
            if audios[i].endswith('.mp3'):
                # print('found mp3', audios[i])
                wav_data, _ = mp3_to_wav(audios[i])
            else:
                wav_data, _ = malaya_speech.load(audios[i], sr = sr)

            if (len(wav_data) / sr) > maxlen:
                # print(f'skipped audio too long {audios[i]}')
                continue

            if len(cleaned_texts[i]) < minlen_text:
                # print(f'skipped text too short {audios[i]}')
                continue

            t = malaya_speech.subword.encode(
                langs[lang - 1], cleaned_texts[i], add_blank=False
            )
            t = np.array(t) + sum(len_vocab[:lang - 1])
            back = np.zeros(shape=(2000,))
            front = np.zeros(shape=(200,))
            wav_data = np.concatenate([front, wav_data, back], axis=-1)

            yield {
                'waveforms': wav_data,
                'targets': t,
                'targets_length': [len(t)],
            }
        except Exception as e:
            print(e)


def preprocess_inputs(example):
    s = featurizer.vectorize(example['waveforms'])
    mel_fbanks = tf.reshape(s, (-1, n_mels))
    length = tf.cast(tf.shape(mel_fbanks)[0], tf.int32)
    length = tf.expand_dims(length, 0)
    example['inputs'] = mel_fbanks
    example['inputs_length'] = length
    example.pop('waveforms', None)
    return example


def get_dataset(
    file,
    batch_size = 3,
    shuffle_size = 20,
    thread_count = 24,
    maxlen_feature = 1800,
):
    def get():
        dataset = tf.data.Dataset.from_generator(
            generate,
            {
                'waveforms': tf.float32,
                'targets': tf.int32,
                'targets_length': tf.int32,
            },
            output_shapes = {
                'waveforms': tf.TensorShape([None]),
                'targets': tf.TensorShape([None]),
                'targets_length': tf.TensorShape([None]),
            },
            args = (file,),
        )
        dataset = dataset.prefetch(tf.contrib.data.AUTOTUNE)
        dataset = dataset.map(
            preprocess_inputs, num_parallel_calls = thread_count
        )
        dataset = dataset.padded_batch(
            batch_size,
            padded_shapes = {
                'inputs': tf.TensorShape([None, n_mels]),
                'inputs_length': tf.TensorShape([None]),
                'targets': tf.TensorShape([None]),
                'targets_length': tf.TensorShape([None]),
            },
            padding_values = {
                'inputs': tf.constant(0, dtype = tf.float32),
                'inputs_length': tf.constant(0, dtype = tf.int32),
                'targets': tf.constant(0, dtype = tf.int32),
                'targets_length': tf.constant(0, dtype = tf.int32),
            },
        )
        return dataset

    return get

In [6]:
dev_dataset = get_dataset('mixed-asr-test.json')()

In [7]:
features = dev_dataset.make_one_shot_iterator().get_next()
features

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


{'targets': <tf.Tensor 'IteratorGetNext:2' shape=(?, ?) dtype=int32>,
 'targets_length': <tf.Tensor 'IteratorGetNext:3' shape=(?, ?) dtype=int32>,
 'inputs': <tf.Tensor 'IteratorGetNext:0' shape=(?, ?, 80) dtype=float32>,
 'inputs_length': <tf.Tensor 'IteratorGetNext:1' shape=(?, ?) dtype=int32>}

In [8]:
training = True

In [9]:
config = malaya_speech.config.conformer_base_encoder_config
config['dropout'] = 0.0
conformer_model = conformer.Model(
    kernel_regularizer = None, bias_regularizer = None, **config
)
decoder_config = malaya_speech.config.conformer_base_decoder_config
decoder_config['embed_dropout'] = 0.0
transducer_model = transducer.rnn.Model(
    conformer_model, vocabulary_size = sum(len_vocab), **decoder_config
)
targets_length = features['targets_length'][:, 0]
v = tf.expand_dims(features['inputs'], -1)
z = tf.zeros((tf.shape(features['targets'])[0], 1), dtype = tf.int32)
c = tf.concat([z, features['targets']], axis = 1)

logits = transducer_model([v, c, targets_length + 1], training = training)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
decoded = transducer_model.greedy_decoder(v, features['inputs_length'][:, 0], training = training)

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
saver = tf.train.Saver(var_list = var_list)
saver.restore(sess, 'asr-base-mixed-stack-conformer-transducer/model.ckpt-1150000')

INFO:tensorflow:Restoring parameters from asr-base-mixed-stack-conformer-transducer/model.ckpt-1150000


In [12]:
import re
from malaya_speech.utils.subword import decode

def decode_multilanguage(row, langs):
    
    if not len(row):
        return ''

    len_vocab = [l.vocab_size for l in langs]

    def get_index_multilanguage(r):
        for i in range(len(langs)):
            sum_v = sum(len_vocab[:i + 1])
            if r < sum(len_vocab[:i + 1]):
                return i, r - sum(len_vocab[:i])

    last_index, v = get_index_multilanguage(row[0])
    d, q = [], [v]
    for r in row[1:]:
        index, v = get_index_multilanguage(r)
        if index != last_index:
            d.append(decode(langs[last_index], q))
            q = [v]
            last_index = index
        else:
            q.append(v)
    if len(q):
        d.append(decode(langs[last_index], q))
    d = re.sub(r'[ ]+', ' ', ' '.join(d)).strip()
    d = d.replace(' lah', 'lah')
    return d

In [13]:
wer, cer = [], []
index = 0
while True:
    try:
        r = sess.run([decoded, features['targets']])
        for no, row in enumerate(r[0]):
            try:
                d = decode_multilanguage(row[row > 0], langs)
                t = decode_multilanguage(r[1][no], langs)
                wer.append(malaya_speech.metrics.calculate_wer(t, d))
                cer.append(malaya_speech.metrics.calculate_cer(t, d))
            except Exception as e:
                print('inside', e)
        index += 1
    except Exception as e:
        print(e)
        break

End of sequence
	 [[node IteratorGetNext (defined at /home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'IteratorGetNext':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/husein/.local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/husein/.local/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/husein/.local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/home/husein/.local/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.6/asyncio/base_events.py", line 438, in run_forever
    self._r

In [14]:
np.mean(wer), np.mean(cer)

(0.24019828699258888, 0.1577375218545968)

In [15]:
for no, row in enumerate(r[0]):
    try:
        d = decode_multilanguage(row[row > 0], langs)
        t = decode_multilanguage(r[1][no], langs)
        print(d)
        print(t)
        print()
    except:
        pass

john so he chose drawn
john so he choose john

